In [1]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# We have to check which Torch version for Xformers (2.3 -> 0.0.27)
# from torch import __version__; from packaging.version import Version as V
# xformers = "xformers==0.0.27" if V(__version__) < V("2.4.0") else "xformers"
# !pip install --no-deps {xformers}
!pip install trl peft accelerate bitsandbytes triton wandb

In [2]:
!pip3 install -U xformers --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 65.9 MB/s eta 0:00:00


In [3]:
import torch

if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [4]:
from unsloth import FastLanguageModel
from unsloth import is_bfloat16_supported
from trl import SFTTrainer
from transformers import TrainingArguments, TextStreamer
from datasets import load_dataset
import torch
import wandb

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


### Load model

In [5]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit",
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",
    "unsloth/gemma-2-2b-bnb-4bit",
]

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/gemma-2-2b",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

==((====))==  Unsloth 2024.11.10: Fast Gemma2 patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth 2024.11.10 patched 26 layers with 26 QKV layers, 26 O layers and 26 MLP layers.


In [ ]:
wandb.login()

In [ ]:
run = wandb.init(
    project='gemma2-2b-sqlgen-unsloth',
    job_type="training",
    config={
               "architecture": "gemma2-2b",
               "dataset": "gretelai/synthetic_text_to_sql",
    }
)

### Prepare Dataset

In [9]:
dataset = load_dataset("gretelai/synthetic_text_to_sql")

README.md:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

In [10]:
def generate_prompt(data_point):
    """Generate input text based on a prompt, task instruction, (context info), and answer.

    :param data_point: dict: Data point
    :return: dict: Data point with the added "prompt" field
    """
    prompt_template = """
    <start_of_turn>user
    You are an intelligent AI specialized in generating SQL queries.
    Your task is to assist users in formulating SQL queries to retrieve specific information from a database.
    Please provide the SQL query corresponding to the given prompt and context:

    Prompt:
    {sql_prompt}

    Context:
    {sql_context}<end_of_turn>
    <start_of_turn>model
    {sql}<end_of_turn>
    """

    prompt_text = prompt_template.format(sql_prompt=data_point["sql_prompt"],sql_context=data_point["sql_context"],sql=data_point["sql"])
    data_point["prompt"] = prompt_text

    return data_point

In [11]:
dataset = dataset['train'].map(generate_prompt)

dataset

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'domain', 'domain_description', 'sql_complexity', 'sql_complexity_description', 'sql_task_type', 'sql_task_type_description', 'sql_prompt', 'sql_context', 'sql', 'sql_explanation', 'prompt'],
    num_rows: 100000
})

In [12]:
print(dataset['prompt'][99])


    <start_of_turn>user
    You are an intelligent AI specialized in generating SQL queries.
    Your task is to assist users in formulating SQL queries to retrieve specific information from a database.
    Please provide the SQL query corresponding to the given prompt and context:

    Prompt:
    How many electric vehicles are there in China and Japan?

    Context:
    CREATE TABLE electric_vehicles (country VARCHAR(50), num_vehicles INT); INSERT INTO electric_vehicles (country, num_vehicles) VALUES ('China', 1140000), ('Japan', 850000);<end_of_turn>
    <start_of_turn>model
    SELECT SUM(num_vehicles) FROM electric_vehicles WHERE country IN ('China', 'Japan');<end_of_turn>
    


In [13]:
dataset = dataset.shuffle(seed=2000)  # Shuffle dataset here
dataset = dataset.map(lambda samples: tokenizer(samples["prompt"]), batched=True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

In [14]:
dataset = dataset.train_test_split(test_size=0.2)
train_data = dataset["train"]
test_data = dataset["test"]

### Train

In [15]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset['train'],
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to='wandb',
    ),
)

max_steps is given, it will override any value given in num_train_epochs


In [16]:
trainer_state = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 80,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 20,766,720
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,1.910200
2,1.826800
3,2.070100
4,1.907900
5,1.601200
6,1.220600
7,1.071300
8,0.871900
9,0.714200
10,0.686900


### Inference

#### stream inference

In [17]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

prompt_template = """
<start_of_turn>user
You are an intelligent AI specialized in generating SQL queries.
Your task is to assist users in formulating SQL queries to retrieve specific information from a database.
Please provide the SQL query corresponding to the given prompt and context:

Prompt:
What is the total production of oil from the onshore fields in the Beaufort Sea?

Context:
CREATE TABLE beaufort_sea_oil_production (field VARCHAR(255), year INT, production FLOAT);<end_of_turn>
<start_of_turn>model
    """

inputs = tokenizer(
[
   prompt_template,
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<bos>
<start_of_turn>user
You are an intelligent AI specialized in generating SQL queries.
Your task is to assist users in formulating SQL queries to retrieve specific information from a database.
Please provide the SQL query corresponding to the given prompt and context:

Prompt:
What is the total production of oil from the onshore fields in the Beaufort Sea?

Context:
CREATE TABLE beaufort_sea_oil_production (field VARCHAR(255), year INT, production FLOAT);<end_of_turn>
<start_of_turn>model
    SELECT SUM(production) FROM beaufort_sea_oil_production WHERE field = 'Beaufort Sea' AND year >= 2020;<end_of_turn>
<start_of_turn>model
    SELECT SUM(production) FROM beaufort_sea_oil_production WHERE field = 'Beaufort Sea' AND year >= 2020;<end_of_turn>
<start_of_turn>model
    SELECT SUM(production) FROM beaufort_sea_oil_production WHERE field = 'Beaufort Sea' AND year >= 2020;<end_of_turn>
<start_of_turn>model
    SELECT SUM(production) FROM beaufort_sea_oil_production WHERE field =


### Saving, loading finetuned models

In [21]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

#### Inference with LoRA

In [20]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model)


inputs = tokenizer(
[
  prompt_template,
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
generated_ids = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
ans = ''
for i in tokenizer.decode(generated_ids[0], skip_special_tokens=True).split('<end_of_turn>')[:2]:
    ans += i

# Extract only the model's answer
model_answer = ans.split("model")[1].strip()

print("Model Answer:\n", model_answer)

<bos>
<start_of_turn>user
You are an intelligent AI specialized in generating SQL queries.
Your task is to assist users in formulating SQL queries to retrieve specific information from a database.
Please provide the SQL query corresponding to the given prompt and context:

Prompt:
What is the total production of oil from the onshore fields in the Beaufort Sea?

Context:
CREATE TABLE beaufort_sea_oil_production (field VARCHAR(255), year INT, production FLOAT);<end_of_turn>
<start_of_turn>model
    SELECT SUM(production) FROM beaufort_sea_oil_production WHERE field = 'Beaufort Sea' AND year >= 2020;<end_of_turn>
<start_of_turn>model
    SELECT SUM(production) FROM beaufort_sea_oil_production WHERE field = 'Beaufort Sea' AND year >= 2020;<end_of_turn>
<start_of_turn>model
    SELECT SUM(production) FROM beaufort_sea_oil_production WHERE field = 'Beaufort Sea' AND year >= 2020;<end_of_turn>
<start_of_turn>model
    SELECT SUM(production) FROM beaufort_sea_oil_production WHERE field =
Model

In [18]:
prompt_template = """
<start_of_turn>user
You are an intelligent AI specialized in generating SQL queries.
Your task is to assist users in formulating SQL queries to retrieve specific information from a database.
Please provide the SQL query corresponding to the given prompt and context:

Prompt:
What is the total production of oil from the onshore fields in the Beaufort Sea?

Context:
CREATE TABLE beaufort_sea_oil_production (field VARCHAR(255), year INT, production FLOAT);<end_of_turn>
<start_of_turn>model
    """

prompt = prompt_template
encodeds = tokenizer(prompt, return_tensors="pt", add_special_tokens=True).input_ids

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
inputs = encodeds.to(device)


# Increase max_new_tokens if needed
generated_ids = model.generate(inputs, max_new_tokens=1000, do_sample=True, temperature = 0.7,pad_token_id=tokenizer.eos_token_id)
ans = ''
for i in tokenizer.decode(generated_ids[0], skip_special_tokens=True).split('<end_of_turn>')[:2]:
    ans += i

# Extract only the model's answer
model_answer = ans.split("model")[1].strip()
print(model_answer)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


SELECT SUM(production) FROM beaufort_sea_oil_production WHERE field IN ('Beaufort-1', 'Beaufort-2');
ganj
    SELECT SUM(production) FROM beaufort_sea_oil_production WHERE field IN ('Beaufort-1', 'Beaufort-2');


In [22]:
wandb.finish()

train/epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇█████
train/grad_norm,▆▇▆▅▅▅█▃▃▂▃▂▂▂▂▂▂▁▁▂▃▁▁▂▂▁▁▃▂▂▁▁▂▂▁▁▁▁▁▂
train/learning_rate,▂▄███▇▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁
train/loss,▇▇█▇▆▄▃▂▂▂▂▂▂▁▁▁▂▁▂▁▁▁▁▁▁▁▂▁▁▁▁▂▁▂▁▁▁▁▁▁
total_flos,1514721418407936.0
train/epoch,0.006
train/global_step,60
train/grad_norm,0.50682
train/learning_rate,0
train/loss,0.4608
